In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/tensor-network-moa/tensorly/requirements.txt
/kaggle/input/tensor-network-moa/tensorly/.coveragerc
/kaggle/input/tensor-network-moa/tensorly/Makefile
/kaggle/input/tensor-network-moa/tensorly/deploy_docs.sh
/kaggle/input/tensor-network-moa/tensorly/CONTRIBUTING.rst
/kaggle/input/tensor-network-moa/tensorly/setup.py
/kaggle/input/tensor-network-moa/tensorly/.travis.yml
/kaggle/input/tensor-network-moa/tensorly/README.rst
/kaggle/input/tensor-network-moa/tensorly/LICENSE.txt
/kaggle/input/tensor-network-moa/tensorly/setup.cfg
/kaggle/input/tensor-network-moa/tensorly/AUTHORS.rst
/kaggle/input/tensor-network-moa/tensorly/CHANGELOG.md
/kaggle/input/tensor-network-moa/tensorly/.gitignore
/kaggle/input/tensor-network-moa/tensorly/.git/index
/kaggle/input/

In [2]:
!mkdir /kaggle/temp
!cp -r /kaggle/input/tensor-network-moa/tensorly/ /kaggle/temp/
!cd /kaggle/temp/tensorly
!ls
!pip install /kaggle/temp/tensorly

__notebook__.ipynb
Processing /kaggle/temp/tensorly
  Created wheel for tensorly: filename=tensorly-0.5.0-py3-none-any.whl size=144473 sha256=513954b42fded99303cdf9a986f1a9bd2b698923ae9fcc8dca3c30fd33dcd3fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-sw8g7if1/wheels/38/c3/d3/dc77a7593ea8594c8b905b2405c0cbc1341b31aac30fbd80a4
Successfully built tensorly


In [3]:
!cat instalogs.txt

cat: instalogs.txt: No such file or directory


In [4]:
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

data = pd.read_csv("/kaggle/input/lish-moa/train_features.csv", index_col="sig_id")
test_data = pd.read_csv("/kaggle/input/lish-moa/test_features.csv", index_col="sig_id")
targets = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv", index_col="sig_id")

cat_f = ["cp_type", "cp_time", "cp_dose"]
oh = OneHotEncoder()
cats = oh.fit_transform(data[cat_f].values).toarray()

non_cats = data.columns[3:]

train_X = np.concatenate([data[non_cats].values, cats], 1)

train_ind, test_ind = train_test_split(np.arange(train_X.shape[0]), train_size=0.95)

test_cats = oh.transform(test_data[cat_f].values).toarray()

test_X = np.concatenate([test_data[non_cats].values, test_cats], 1)

np.save("/kaggle/temp/train_X.npy", train_X[train_ind])
np.save("/kaggle/temp/train_Y.npy", targets.values[train_ind])
np.save("/kaggle/temp/val_X.npy", train_X[test_ind])
np.save("/kaggle/temp/val_Y.npy", targets.values[test_ind])
np.save("/kaggle/temp/testing_X.npy", test_X)

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from tqdm.notebook import tqdm

import torch
import numpy as np
from torch import nn
import tensorly as tl
from tqdm import tqdm
import matplotlib.pyplot
from torch.optim import Adam
from matplotlib import rcParams
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import StepLR

import tensorly as tl
from tensorly.random import random_tucker
from tensorly.tucker_tensor import tucker_to_tensor


from torch.utils.data import Dataset

#tl.set_backend('pytorch')


import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from tqdm.notebook import tqdm

import tensorly as tl
from tensorly.random import random_tucker
from tensorly.tucker_tensor import tucker_to_tensor


from torch.utils.data import Dataset

#tl.set_backend('pytorch')


class NeuralTensorLayer(torch.nn.Module):
    
    """
    This is the class for 
    """
    
    def __init__(self, order, input_dim, output_dim, rank_tucker=-1,
                 initializer=torch.nn.init.xavier_uniform):
        
        super(NeuralTensorLayer, self).__init__()
        
        self.order = order
        self.rank_tucker = rank_tucker
        
        if order > 3 or order < 1:
            raise Exception('Order must be in range [1, 3]')
            
        if rank_tucker != -1 and rank_tucker < 1:
            raise Exception('Tucker rank must be -1 or greater than 0 integer')
            
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        self.bias = nn.Parameter(torch.zeros((1, output_dim)), requires_grad=True)
        initializer(self.bias)
        
        self.myparameters = torch.nn.ParameterList([self.bias])
        
        self.order1_tens = self.initialize_n_order_tensor(1, initializer)
        
        if order >= 2:
            self.order2_tens = self.initialize_n_order_tensor(2, initializer)
            
        if order == 3:
            self.order3_tens = self.initialize_n_order_tensor(3, initializer)
        
    # initialize tensor in full or in decomposed form and register it as parameter
    def initialize_n_order_tensor(self, order, initializer):
        
        if self.rank_tucker >= 1:
            
            dim_list = [self.input_dim] * order + [self.output_dim]
            tens_core, factors = random_tucker(dim_list, self.rank_tucker)
            tens_core = nn.Parameter(tens_core, requires_grad=True)
            factors = [nn.Parameter(fact, requires_grad=True) for fact in factors]
            
            self.myparameters.append(tens_core)
            for fact in factors:
                self.myparameters.append(fact)
                
            return (tens_core, factors)
            
        else:
            
            dim_list = [self.input_dim] * order + [self.output_dim]
            var = nn.Parameter(torch.zeros(dim_list), requires_grad=True)
            initializer(var)
            self.myparameters.append(var)
            
            return var

    def compute_result_for_vec(self, core, factor_inp, last_factor): # result dim (1, 1)
        result = core
        for i in range(len(factor_inp)):
            result = tl.tenalg.mode_dot(result, factor_inp[i], i)
        result = result.view(1, -1).mm(torch.transpose(last_factor, 0, 1))
        return result.view(-1)

    def mode_n_dot_accelerated(self, core, factors, input):

        new_factors = [torch.transpose(factors[i], 0, 1).mm(input) for i in range(len(factors) - 1)]

        return torch.stack([
                            self.compute_result_for_vec(core, 
                                                        [new_factors[k][:, i] for k in range(len(factors) - 1)], factors[-1]) 
                            for i in range(input.shape[1])
                            ], dim=0)
        
    def forward(self, X, transposed=False):
        
        #X = torch.Tensor(X)
        
        if self.rank_tucker == -1:
            result = torch.addmm(self.bias, X, self.order1_tens)
        else:
            result = torch.addmm(self.bias, X, tucker_to_tensor(self.order1_tens))
        
        if self.order >= 2:
            
            if self.rank_tucker == -1:      
                acc = tl.tenalg.mode_dot(self.order2_tens, X, 0)
            else:
                acc = tl.tenalg.mode_dot(tucker_to_tensor(self.order2_tens), X, 0)

            acc = tl.tenalg.mode_dot(acc, X, 1)
            result += torch.einsum('iik->ik', acc)
        
        if self.order == 3:
             
            if self.rank_tucker == -1:      
                acc = tl.tenalg.mode_dot(self.order3_tens, X, 0)
            else:
                acc = tl.tenalg.mode_dot(tucker_to_tensor(self.order3_tens), X, 0)
            
            acc = tl.tenalg.mode_dot(acc, X, 1)
            acc = tl.tenalg.mode_dot(acc, X, 2)
            result += torch.einsum('iiik->ik', acc)
        
        return tl.reshape(result, (X.shape[0], self.output_dim))

    def get_orthogonality_loss(self):

        if self.rank_tucker == -1:
            return 0

        loss = 0

        for fact in self.order1_tens[1]:
            loss += torch.sum((tl.dot(fact.T, fact) - torch.eye(fact.shape[1]).cuda()) ** 2)
        
        if self.order >= 2:
            
            for fact in self.order2_tens[1]:
                loss += torch.sum((tl.dot(fact.T, fact) - torch.eye(fact.shape[1]).cuda()) ** 2)
        
        if self.order == 3:
             
            for fact in self.order3_tens[1]:
                loss += torch.sum((tl.dot(fact.T, fact) - torch.eye(fact.shape[1]).cuda()) ** 2)

        return loss


class MOA_set(Dataset):
    
    def __init__(self, fn_X, fn_Y=None, cuda=True):
        self.X = np.load(fn_X)
        self.Y = None if not fn_Y else np.load(fn_Y)
        self.Yn = True if not fn_Y else False
        self.cuda = cuda
        
    def __len__(self):
        return(self.X.shape[0])
    
    def __getitem__(self, ind):
        x = torch.from_numpy(self.X[ind]).type(torch.FloatTensor)
        if self.cuda:
            x = x.cuda()
        if not self.Yn:
            y = torch.from_numpy(self.Y[ind]).type(torch.FloatTensor)
            if self.cuda:
                y = y.cuda()
            return x, y
        else:
            return x




#NN = torch.nn.Sequential(NeuralTensorLayer(2, input_dim, 2, rank_tucker=5), 
 #                        nn.BatchNorm1d(2),
  #                       nn.Softmax(dim=-1))
#loss = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(NN.parameters(), lr=0.001)


# In[ ]:

class MOA_set(Dataset):
    
    def __init__(self, fn_X, fn_Y=None, cuda=True):
        self.X = np.load(fn_X)
        self.Y = None if not fn_Y else np.load(fn_Y)
        self.Yn = True if not fn_Y else False
        self.cuda = cuda
        
    def __len__(self):
        return(self.X.shape[0])
    
    def __getitem__(self, ind):
        x = torch.from_numpy(self.X[ind]).type(torch.FloatTensor)
        if self.cuda:
            x = x.cuda()
        if not self.Yn:
            y = torch.from_numpy(self.Y[ind]).type(torch.FloatTensor)
            if self.cuda:
                y = y.cuda()
            return(x, y)
        else:
            return(x)




class TensorNet(nn.Module): 
    
    def __init__(
        self, order, input_dim, output_dim, rank_tucker=5
    ):
        super(TensorNet, self).__init__()
        self.tensor = NeuralTensorLayer(
            order, input_dim, output_dim, rank_tucker=rank_tucker
        )
        self.bn = nn.BatchNorm1d(output_dim)
        #self.l = nn.Linear(tensor_dim, output_dim)
        self.s = nn.Sigmoid()
        
    def forward(self, x):
        return(self.s(self.bn(self.tensor(x))))
    
    def predict(self, loader, loss=None, train=True, verbose=False, orth_alpha=None):
        Y_hat = []
        mean_loss = 0
        ld = tqdm(loader) if verbose else loader
        for i,a in enumerate(ld):
            d = a[0] if train else a
            c_Y_hat = self.forward(d)
            c_y_hat = c_Y_hat.cpu().data.numpy()
            Y_hat.append(c_y_hat)
            if train:
                loss_val = loss(c_Y_hat, a[1])
                if orth_alpha:
                    loss_val += self.tensor.get_orthogonality_loss()*orth_alpha
                mean_loss += loss_val.cpu().data.numpy()
        return(np.concatenate(Y_hat), mean_loss/i)
    
    def fit(
        self, loader, loss, optimizer, scheduler, n_iter, 
        val_loader, metrics, print_every=10, orth_alpha=0.1, fit_orthogonality=True
    ):
        history = {a: [] for a in metrics}
        history["train_loss"] = []
        history["val_loss"] = []
        for j in tqdm(np.arange(n_iter)):
            mean_loss = 0
            self.train()
            for i,(batch_X, batch_Y) in enumerate(loader):
                optimizer.zero_grad()
                Y_hat = self.forward(batch_X)
                loss_val = loss(Y_hat, batch_Y)
                if fit_orthogonality:
                    loss_val += self.tensor.get_orthogonality_loss()*orth_alpha
                loss_val.backward()
                optimizer.step()
                mean_loss += loss_val.cpu().data.numpy()
            history["train_loss"].append(mean_loss/i)
            self.eval()
            val_Y_hat, val_loss = self.predict(val_loader, loss, orth_alpha=orth_alpha)
            history["val_loss"].append(val_loss)
            val_Y = np.concatenate([a[1].cpu().data.numpy() for a in val_loader])
            for m in metrics:
                history[m].append(metrics[m](val_Y, val_Y_hat))
            if (j+1) % print_every == 0:
                print("epoch#"+str(j))
                print("train loss", history["train_loss"][-1])
                print("val loss", history["val_loss"][-1])
                for m in metrics:
                    print(m, history[m][-1])
            clip_grad_norm_(self.parameters(), 0.3)
            scheduler.step()
        return(history)

In [6]:
np.random.seed(1337)
torch.manual_seed(1377)

tl.set_backend('pytorch')

train_set = MOA_set("/kaggle/temp/train_X.npy", "/kaggle/temp/train_Y.npy")
val_set = MOA_set("/kaggle/temp/val_X.npy", "/kaggle/temp/val_Y.npy")
test_set = MOA_set("/kaggle/temp/testing_X.npy")

BATCH_SIZE = 32

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

model = TensorNet(2, 879, 206).cuda()

loss = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.9)

def colwise_logloss(Y, Y_hat):
    return(-(Y * np.log(Y_hat+10**-6) + (1 - Y) * np.log(1 - Y_hat+10**-6)).mean())

history = model.fit(
    train_loader, loss, optimizer, scheduler, 60, val_loader, 
    {"CWLL": colwise_logloss}, 5
)

import pandas as pd

test_predictions = model.predict(test_loader, train=False, verbose=True)[0]

out_df = pd.DataFrame(test_predictions)

out_df.columns = targets.columns
out_df.index = test_data.index

out_df.to_csv("/kaggle/working/submission.csv")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:69: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


epoch#4
train loss 290.48620452016337
val loss 186.85004899308487
CWLL 0.08955539
epoch#9
train loss 5.320567772003477
val loss 3.432651867737641
CWLL 0.0313432
epoch#14
train loss 0.04574296354097195
val loss 0.030810967440137994
CWLL 0.021235729
epoch#19
train loss 0.02018180725932459
val loss 0.020036382298614527
CWLL 0.019451756
epoch#24
train loss 0.019686692072442672
val loss 0.0198943653219455
CWLL 0.019338604
epoch#29
train loss 0.019215418685428143
val loss 0.019094935819708014
CWLL 0.018577501
epoch#34
train loss 0.018627192009841923
val loss 0.018471083855508146
CWLL 0.0179552
epoch#39
train loss 0.017704983755006922
val loss 0.017308160989872506
CWLL 0.016773773
epoch#44
train loss 0.017033834895514816
val loss 0.016743789024248317
CWLL 0.016199045
epoch#49
train loss 0.016628550703270576
val loss 0.016651161317084287
CWLL 0.01608396
epoch#54
train loss 0.016292054757539174
val loss 0.016371447819511633
CWLL 0.015834786
epoch#59
train loss 0.016068619586231857
val loss 0.01

In [7]:
out_df.to_csv("submission.csv")